In [1]:
import numpy as np
import mygrad
import mygrad.functional as F
import mygrad.nn as nn

In [2]:
from sklearn.datasets import fetch_california_housing

X,y = fetch_california_housing(return_X_y=True)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [4]:
X_train.shape

(15480, 8)

In [5]:
class FCNet(nn.Module):
    def __init__(self, name, init="xavier_normal"):
        super(FCNet, self).__init__(name)
        
        self.linear1 = nn.Linear(f"{name}.linear1", 8, 32, init=init)
        self.tanh1 = F.tanh()
        self.linear2 = nn.Linear(f"{name}.linear2", 32, 1, init=init)
        
    def forward(self, X):
        X = self.linear1.forward(X)
        X = self.tanh1.forward(X)
        X = self.linear2.forward(X)
        
        return X
    
    def parameters(self, ):
        return self.linear1.parameters() + self.linear2.parameters()

In [6]:
history = []

model = FCNet("fcnet")

loss_fn = F.mse_loss()
optim = nn.Adam(model.parameters())

for i in range(500):
    ix = np.random.randint(0, len(X_train), 256)
    x_batch = mygrad.value(X_train[ix], "x_batch")
    y_batch = mygrad.value(y_train[ix], "y_batch")
    
    y = model.forward(x_batch)
    loss = loss_fn.forward(y, y_batch)
    
    loss_fn.backward()   #Slightly different from torch
    optim.step()
    
    optim.zero_grad()
    
    history.append(loss.data)

    if i % 10 == 0:
        print("step #%i | mean loss = %.3f" % (i, np.mean(history[-10:])))
    
    

Creating Function of type <class 'mygrad.functional.function.Matmul'> with name matmul_0
Creating Function of type <class 'mygrad.functional.function.Add'> with name add_0
Creating Function of type <class 'mygrad.functional.function.Tanh'> with name tanh_0
Creating Function of type <class 'mygrad.functional.function.Matmul'> with name matmul_1
Creating Function of type <class 'mygrad.functional.function.Add'> with name add_1
Creating Function of type <class 'mygrad.functional.function.MSELoss'> with name mse_loss_0
fcnet.linear1.W (8, 32) (8, 32)
fcnet.linear1.b (32,) (32,)
fcnet.linear2.W (32,) (32,)


C:\Users\User\Documents\mygrad\mygrad\functional\function.py:200: RuntimeWarning: overflow encountered in exp
  t = 1 - 2 / (1 + np.exp(2 * x.data))


ValueError: non-broadcastable output operand with shape (32,1) doesn't match the broadcast shape (32,32)

In [ ]:
X_train = mygrad.value(X_train, "X_train", requires_grad=False)
X_test = mygrad.value(X_test, "X_test", requires_grad=False)
y_train = mygrad.value(y_train, "y_train", requires_grad=False)
y_test = mygrad.value(y_test, "y_test", requires_grad=False)

y_train_pred = model.forward(X_train)
y_test_pred = model.forward(X_test)

train_loss = loss_fn.forward(y_train_pred, y_batch)
test_loss = loss_fn.forward(y_train_pred, y_batch)

print(f"Train MSE: {train_loss.data}")
print(f"Test MSE: {test_loss.data}")